<div class="contentcontainer med left" style="margin-left: -50px;">
<dl class="dl-horizontal">
  <dt>Description</dt> <dd> Game of Life Simulator quickstart guide</dd>
  <dt>Author</dt> <dd>Jean-Luc Stevens</dd>
  <dt>HoloViews</dt> <dd>>1.6.2</dd>
  <dt>Python</dt> <dd>2.7/3.3+</dd>
</dl>
</div>

This quickstart is designed to demonstrate how an open-ended simulation can be integrated with HoloViews, starting from HoloViews 1.7. Using [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) as an example of a system to simulate, this quickstart guide shows how the streams system supports dimensioned streams to keep track of simulation time. This notebook uses core HoloViews together with the matplotlib backend and also requires [scipy](https://www.scipy.org/) which can be installed using conda with:

```bash
conda install scipy
```


In [ ]:
import holoviews as hv
from holoviews import streams
import numpy as np
hv.notebook_extension()

# Conway's Game of Life

[Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) is a famous example of a 2D cellular automata. The simulations runs on a grid, consisting of cells that can be in one of two states: 'alive' (on) or 'dead' (off). The simulation is updated in discrete steps and the next state of a cell depends on the state of its eight neighbors:

* Any live cell with fewer than two live neighbours dies ('under-population')
* Any live cell with two or three live neighbours survives and lives on to the next generation.
* Any live cell with more than three live neighbours dies ('over-population').
* Any dead cell with exactly three live neighbours becomes a live cell ('reproduction').

Using the ``convolve2d`` function from ``scipy``, we can use a very simple convolution kernel to cover the eight neighbors and effectively count the surrounding live cells. The details are not too important, what matters is the core of the Game of Life can be written as follows to compute the next state of boolean array ``X``:

In [ ]:
from scipy.signal import convolve2d

def single_step(X):
    "Compute the next step in the Game of Life using a 2D convolution"
    nbrs_count = convolve2d(X, np.ones((3, 3)), mode='same', boundary='wrap') - X
    return (nbrs_count == 3) | (X & (nbrs_count == 2))

To test it, we will declare a board containing the famous [glider pattern](https://en.wikipedia.org/wiki/Glider_(Conway%27s_Life)) to check this function correctly advances by one simulation step:

In [ ]:
glider = [[1, 0, 0], [0, 1, 1], [1, 1, 0]]
board = hv.Image(np.pad(np.array(glider, dtype=int), 5, 'constant'))
board + hv.Image(single_step(board.data))

Now this seems to work, lets build the simplest version of our simulator by declaring the following class:

In [ ]:
class GameOfLifeSimple(object):
    """
    Simple simulator of Conway's Game of Life that starts with a 
    given starting pattern (a simple glider by default) and some
    specified level of padding around it.
    """
    
    def __init__(self, pattern=np.array(glider, dtype=int), padding=5):
        self.time = 0
        self.board = hv.Image(np.pad(pattern, padding, 'constant'))
        
    def step(self):
        "Advance the state by one step, returning a new Image "
        data = single_step(self.board.data)
        self.board.data = data
        self.time += 1
        return hv.Image(data)
    
    def run_until(self, time):
        "Advance the simulation to the specified time"
        times = range(self.time, time)
        for t in times:
            state = self.step()
        return state if times else hv.Image(self.board.data)

Using this class, we can use the ``run_until`` method to move forward nine steps at once in simulation time, making it clear that the glider is moving diagonally downwards to the right:

In [ ]:
GoL = GameOfLifeSimple()
board  + GoL.run_until(9)

Using a ``HoloMap`` is one way to interact with the simulation although we are limited in how many steps can be sampled before we use up too much memory. The advantage of HoloMaps is that they can be used after exporting the notebook to HTML. Let's make a HoloMap showing our glider over 10 steps:

In [ ]:
GoL = GameOfLifeSimple()
hv.HoloMap({t:GoL.run_until(t) for t in range(10)}, kdims=[hv.Dimension('Time', type=int)])

# Using DynamicMap

The Game of Life is a simulation that is unbounded and can be run forever, even if the state of the world eventually reaches a steady state. For this reason, both ``HoloMaps`` and bounded ``DynamicMaps`` are not ideal as we want the ability to leave the simulation running for as long as we like.

In HoloViews 1.7, you can use the streams system together with ``DynamicMap`` to define and open-ended simulation. In the following cell, a custom ``Stream `` is defined and used to build an open-ended simulation in the for of a ``DynamicMap`` called ``sim1``:

In [ ]:
import param

class SimulationTime(streams.Stream):
    
    time = param.Integer(default=0, bounds=(0,None))

GoL = GameOfLifeSimple()
sim1 = hv.DynamicMap(GoL.run_until, kdims=[], streams=[SimulationTime()])
sim1

For more information about streams, see the [streams quickstart ](streams.ipynb) guide. We can now advance ``sim1`` using the ``event`` method to make our glider wrap once around its tiny world:

In [ ]:
for t in range(1, 53):
    sim1.event(time=t)

This approach of declaring a ``DynamicMap`` without any ``kdims`` and using streams is great for visualizing live data without holding onto any state. The problem with this is that we have no history and cannot revisit previous state as the ``DynamicMap`` has no elements cached:

In [ ]:
sim1.keys()

This also means we cannot make a ``HoloMap`` out of such ``DynamicMaps`` as they have no key dimensions. You can declare stream parameters as key dimensions, using *dimensioned streams*.

# Dimensioned streams

To use dimensioned streams, simply declare the stream parameter (in this instance, this is the parameter ``time`` of ``SimulationTime``) in the list of ``kdims``:

In [ ]:
GoL = GameOfLifeSimple()
sim2 = hv.DynamicMap(GoL.run_until, kdims=['time'], streams=[SimulationTime()], cache_size=10)
sim2

Note that this time, when we update the time stream, the relevant time is shown in the title because it has now declared as a ``kdim``:

In [ ]:
for t in range(53):
    sim2.event(time=t)

When declaring the ``DynamicMap``, a value of ``cache_size=10`` was specified which means we now have the past ten values of the ``'time'`` dimension:

In [ ]:
sim2.keys()

As we now have keys for our cache, we can now turn the last ten steps of the simulation into a ``HoloMap`` that can be exported with the notebook:

In [ ]:
hv.HoloMap(sim2)

# Rich Simulations

The current simulation example is quite simple as there is only one type of state to worry about (the state of the cells in the world). Now we will extend this simple simulator to have more than one type of state we can visualize at a given time:

In [ ]:
class GameOfLife(GameOfLifeSimple):
    
    def __init__(self, **kwargs):
        super(GameOfLife, self).__init__(**kwargs)
        self._history = hv.Image(np.ones(self.board.data.shape) * self.time)
        
    def step(self):
        state = super(GameOfLife, self).step()
        self._history.data = np.where(state.data==1, state.data*self.time, self._history.data)
        return state
    
    def history_until(self, time):
        self.run_until(time)
        return hv.Image(self._history.data)

The 'history' is a view of the world over time, where cells are marked with the most recent time when an 'alive' cell was present. To illustrate, here is the world state after ten steps next to this view of the history:

In [ ]:
%%opts Image {+axiswise}
GoL = GameOfLife()
GoL.run_until(10) + GoL.history_until(10)

Now using the same ``SimulationTime`` stream class and a single instance ``time``, we can advance the state of our simulator while visualizing multiple types of state at once, using the usual HoloViews ``+`` and ``*`` operators. For instance, after defining two ``DynamicMaps`` called ``sim3`` and ``history``, we can do:

In [ ]:
%%opts Image {+axiswise}
GoL = GameOfLife()
time = SimulationTime()
sim3 = hv.DynamicMap(GoL.run_until, kdims=['time'], streams=[time])
history = hv.DynamicMap(GoL.history_until, kdims=['time'], streams=[time])
sim3 + history

Which we can advance as follows:

In [ ]:
for t in range(53):
    sim3.event(time=t)

Note that we can pick either of our ``DynamicMaps`` to advance time on as both are sharing the same ``time`` stream parameter.

# A final example

To show how using dimensioned streams allows you to run open-ended simulations, here is a visualization of the Game of Life state next to its history starting with a larger, randomized initial state:

In [ ]:
%%opts Image {+axiswise}
pattern = (np.random.rand(100,100) < 0.5).astype(int)
GoL = GameOfLife(pattern=pattern, padding=0)
time = SimulationTime()
sim4_history = hv.DynamicMap(GoL.history_until, kdims=['time'], streams=[time], cache_size=20)
sim4 = hv.DynamicMap(GoL.run_until, kdims=['time'], streams=[time], cache_size=20)
sim4 + sim4_history

We can now advance 100 steps while keeping track of the most recent twenty frames in a ``HoloMap``:

In [ ]:
for t in range(101):
    sim4.event(time=t)
    
hv.HoloMap(sim4)

Then then we can continue to advance the simulation another 100 steps in the knowledge that we always have the last twenty frames available if we wish to inspect them:

In [ ]:
for t in range(101, 201):
    sim4.event(time=t)